In [0]:
storageAccountName = 'equitymarketstorage'
storageAccountAccessKey = '32Q+pSDygN/vBS/+2OPfjXJrW9+T4gogsOJY3mEob/GON/Hls27FX1ZaxbBxEjsmA+/+XJn1AXkFj2dUSrOeWA=='
blobContainerName = 'firstcontainer'


if not any(mount.mountPoint == '/mnt/FileStore/MountFolder/' for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
    source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
    mount_point = "/mnt/FileStore/MountFolder/",
    extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
  )
  except Exception as e:
    print("already mounted. Try to unmount first")

display(dbutils.fs.ls("dbfs:/mnt/FileStore/MountFolder/data"))

path,name,size
dbfs:/mnt/FileStore/MountFolder/data/.DS_Store,.DS_Store,6148
dbfs:/mnt/FileStore/MountFolder/data/csv/,csv/,0
dbfs:/mnt/FileStore/MountFolder/data/data.parquet/,data.parquet/,0
dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet/,data_csv.parquet/,0
dbfs:/mnt/FileStore/MountFolder/data/data_json.parquet/,data_json.parquet/,0
dbfs:/mnt/FileStore/MountFolder/data/json/,json/,0


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType ,LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DateType, TimestampType, DecimalType
import datetime

common_event = StructType().add("trade_dt",DateType(),True)\
                    .add("rec_type", StringType(), True)\
                    .add("symbol", StringType(), True)\
                    .add("exchange", StringType(), True)\
                    .add("event_tm", TimestampType(),True)\
                    .add("event_seq_nb", IntegerType(), True)\
                    .add("arrival_tm", TimestampType(), True)\
                    .add("trade_pr", DecimalType(), True)\
                    .add("bid_pr", DecimalType(), True)\
                    .add("bid_size", IntegerType(), True)\
                    .add("ask_pr", DecimalType(), True)\
                    .add("ask_size", IntegerType(), True)\
                    .add("partition", StringType(), True)

In [0]:
from typing import List
from decimal import Decimal
def parse_csv(line:str):
  record = line.split(",")
  try:
    trade_dt= datetime.datetime.strptime(record[0], '%Y-%m-%d')
    rec_type = record[2]
    symbol = record[3]
    exchange = record[6]
    event_tm=datetime.datetime.strptime(record[4], '%Y-%m-%d %H:%M:%S.%f')
    event_seq_nb = int(record[5])
    arrival_tm=datetime.datetime.strptime(record[1], '%Y-%m-%d %H:%M:%S.%f')

    if rec_type == "T":
      trade_pr = Decimal(record[7])
      event = (trade_dt, rec_type, symbol, exchange, event_tm, event_seq_nb, arrival_tm, trade_pr, Decimal('0.0'), 0, Decimal('0.0'), 0, "T")
      return event
    elif rec_type == "Q":
      bid_pr = Decimal(record[7])
      bid_size = int(record[8])
      ask_pr = Decimal(record[9])
      ask_size = int(record[10])
      event = (trade_dt, rec_type, symbol, exchange, event_tm, event_seq_nb, arrival_tm, Decimal('0.0'), bid_pr, bid_size, ask_pr, ask_size, "Q")
      return event
          
  except Exception as e:
      return ('','','','','','','','','','','','','B')

In [0]:
raw=spark.sparkContext.textFile("dbfs:/mnt/FileStore/MountFolder/data/csv/*/NYSE/*.txt")
parsed = raw.map(lambda line: parse_csv(line))
data = spark.createDataFrame(parsed, schema=common_event)

data.write.partitionBy("partition").mode("overwrite").parquet("dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet")

In [0]:
display(data.filter(data.partition=='B'))

trade_dt,rec_type,symbol,exchange,event_tm,event_seq_nb,arrival_tm,trade_pr,bid_pr,bid_size,ask_pr,ask_size,partition


In [0]:
common_event_json = StructType().add("trade_dt",DateType(),True)\
                    .add("file_tm", TimestampType(), True)\
                    .add("event_type", StringType(), True)\
                    .add("symbol", StringType(), True)\
                    .add("event_tm", DateType(),True)\
                    .add("execution_id", StringType(), True)\
                    .add("event_seq_nb", IntegerType(), True)\
                    .add("exchange", StringType(), True)\
                    .add("bid_pr", DecimalType(), True)\
                    .add("bid_size", IntegerType(), True)\
                    .add("ask_pr", DecimalType(), True)\
                    .add("ask_size", IntegerType(), True)\
                    .add("partition", StringType(), True)

In [0]:
import json


def parse_json(line:str):
  record = json.loads(line)
  try:
      trade_dt = datetime.datetime.strptime(record['trade_dt'], '%Y-%m-%d')
      file_tm = datetime.datetime.strptime(record['file_tm'], '%Y-%m-%d %H:%M:%S.%f')
      event_type = record['event_type']
      symbol = record['symbol']
      exchange = record['exchange']
      event_tm = datetime.datetime.strptime(record['event_tm'], '%Y-%m-%d %H:%M:%S.%f')
      event_seq_nb = int(record['event_seq_nb'])

      if event_type == 'T':
        execution_id = record['execution_id']
        price = Decimal(record['price'])
        size = int(record['size'])
        event = [trade_dt, file_tm, event_type, symbol, event_tm, execution_id, event_seq_nb, exchange, price, size, Decimal('0.0'), 0, 'T']
        return event
      elif event_type == 'Q':
        bid_pr = Decimal(record['bid_pr'])
        bid_size = int(record['bid_size'])
        ask_pr = Decimal(record['ask_pr'])
        ask_size = int(record['ask_size'])
        event = [trade_dt, file_tm, event_type, symbol, event_tm, "0", event_seq_nb, exchange, bid_pr, bid_size, ask_pr, ask_size, 'Q']
        return event

  except Exception as e:
      return ('','','','','','','','','','','','','B')

In [0]:
raw_json=spark.sparkContext.textFile("dbfs:/mnt/FileStore/MountFolder/data/json/*/NASDAQ/*.txt")
parsed = raw_json.map(lambda line: parse_json(line))
data = spark.createDataFrame(parsed, schema=common_event_json)

data.write.partitionBy("partition").mode("overwrite").parquet("dbfs:/mnt/FileStore/MountFolder/data/data_json.parquet")

In [0]:
display(data)

trade_dt,file_tm,event_type,symbol,event_tm,execution_id,event_seq_nb,exchange,bid_pr,bid_size,ask_pr,ask_size,partition
2020-08-05,2020-08-05T09:30:00.000+0000,Q,SYMA,2020-08-05,0,1,NASDAQ,76,100,78,100,Q
2020-08-05,2020-08-05T09:30:00.000+0000,Q,SYMA,2020-08-05,0,2,NASDAQ,75,100,76,100,Q
2020-08-05,2020-08-05T09:30:00.000+0000,Q,SYMA,2020-08-05,0,3,NASDAQ,79,100,81,100,Q
2020-08-05,2020-08-05T09:30:00.000+0000,Q,SYMA,2020-08-05,0,4,NASDAQ,75,100,76,100,Q
2020-08-05,2020-08-05T09:30:00.000+0000,Q,SYMA,2020-08-05,0,5,NASDAQ,77,100,78,100,Q
2020-08-05,2020-08-05T09:30:00.000+0000,Q,SYMA,2020-08-05,0,6,NASDAQ,77,100,78,100,Q
2020-08-05,2020-08-05T09:30:00.000+0000,Q,SYMA,2020-08-05,0,7,NASDAQ,76,100,77,100,Q
2020-08-05,2020-08-05T09:30:00.000+0000,Q,SYMA,2020-08-05,0,8,NASDAQ,77,100,78,100,Q
2020-08-05,2020-08-05T09:30:00.000+0000,Q,SYMA,2020-08-05,0,9,NASDAQ,77,100,78,100,Q
2020-08-05,2020-08-05T09:30:00.000+0000,T,SYMA,2020-08-05,EX-10,10,NASDAQ,78,509,0,0,T
